In [1]:
import os 
import sys
import numpy as np
import argparse
from scipy.spatial.transform import Rotation as R 
# add catkin_ws context 
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.true_grounding_env import TrueGroundingEnv
from src.config import cfg_tabletop
import rospy 
import rospkg
import jupyros as jr

from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [2]:
########################################
# initialize environment
########################################
rospy.init_node('eval_code', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')

# setup environment
env = TrueGroundingEnv(cfg_tabletop)
# env.reset()


[DEBUG] [1702272761.538526, 0.000000]: init_node, name[/eval_code], pid[4100086]
[DEBUG] [1702272761.540126, 0.000000]: binding to 0.0.0.0 0
[DEBUG] [1702272761.541535, 0.000000]: bound to 0.0.0.0 35993
[DEBUG] [1702272761.542083, 0.000000]: connecting to junting-PC 57917
[DEBUG] [1702272761.543612, 0.000000]: ... service URL is rosrpc://junting-PC:35993
[DEBUG] [1702272761.545246, 0.000000]: [/eval_code/get_loggers]: new Service instance
[DEBUG] [1702272761.548796, 579.535000]: ... service URL is rosrpc://junting-PC:35993
[DEBUG] [1702272761.550149, 579.537000]: [/eval_code/set_logger_level]: new Service instance
[INFO] [1702272761.700874, 579.670000]: camera_left: Waiting for camera_left/color/camera_info...
[DEBUG] [1702272761.702085, 579.671000]: connecting to junting-PC 37585
[DEBUG] [1702272761.703232, 579.672000]: connecting to junting-PC 57917
[DEBUG] [1702272761.718603, 579.686000]: connecting to junting-PC 57917
[DEBUG] [1702272761.721860, 579.689000]: connecting to junting-P

[DEBUG] [1702272761.752522, 579.718000]: connecting to junting-PC 57917
[INFO] [1702272762.235990, 580.171000]: camera_left: camera_left/color/camera_info received!
[INFO] [1702272762.238053, 580.172000]: camera_left: camera_left/depth/camera_info received!
[DEBUG] [1702272762.245301, 580.180000]: connecting to junting-PC 57917
[INFO] [1702272762.248141, 580.182000]: camera_right: Waiting for camera_right/color/camera_info...
[DEBUG] [1702272762.251431, 580.186000]: connecting to junting-PC 55655
[DEBUG] [1702272762.254281, 580.189000]: connecting to junting-PC 57917
[DEBUG] [1702272762.256005, 580.190000]: connecting to junting-PC 55655
[INFO] [1702272762.773309, 580.686000]: camera_right: camera_right/color/camera_info received!
[INFO] [1702272762.774751, 580.687000]: camera_right: camera_right/depth/camera_info received!
[DEBUG] [1702272762.780833, 580.693000]: connecting to junting-PC 57917
[DEBUG] [1702272762.783682, 580.695000]: connecting to junting-PC 37631
[INFO] [1702272762.7

In [3]:
# 3D fusion 
from src.perception.scene_manager import SceneManager
sensor_data = env.get_sensor_data()
sensor_data['detections_list'] = [{},{},{}]

scene_manager = SceneManager()
scene_manager.update_fusion(sensor_data)

In [4]:
# get cropped point cloud
drawer_bbox = env.get_3d_bbox('cabinet.drawer0')
drawer_pcd = scene_manager.scene_tsdf_full.crop_cloud(
    crop_center=(drawer_bbox[:3] + drawer_bbox[3:]) / 2,
    crop_size=(drawer_bbox[3:] - drawer_bbox[:3])
)
cabinet_bbox = env.get_3d_bbox('cabinet')
cabinet_pcd = scene_manager.scene_tsdf_full.crop_cloud(
    crop_center=(cabinet_bbox[:3] + cabinet_bbox[3:]) / 2,
    crop_size=(cabinet_bbox[3:] - cabinet_bbox[:3])
)


In [5]:
import open3d as o3d 
# o3d.io.write_point_cloud(os.path.join(pkg_root, 'data', 'drawer.pcd'), drawer_pcd)
# o3d.io.write_point_cloud(os.path.join(pkg_root, 'data', 'cabinet.pcd'), cabinet_pcd)

In [6]:
env.parse_adaptive_shape_grasp_pose("apple")

[INFO] [1702272769.572470, 586.685000]: Sending perception data to grasp detection service
[DEBUG] [1702272769.575478, 586.701000]: connecting to junting-PC 41753


position: 
  x: -0.2738809883594513
  y: -0.22210144996643066
  z: 1.0694431066513062
orientation: 
  x: 0.6993529206633681
  y: -0.6677615559119899
  z: 0.1843941981573167
  w: 0.17606469405174707

In [ ]:
# Load files & data 
def filter_drawer0(processed_data, raw_data, mask):
    """
    Filter tasks involving interacting with drawer 0, since it is too far from the robot.
    """

    for i, data in enumerate(processed_data):
        if mask[i] == 0: # already filtered in previous filters 
            continue
        full_query = data['query']
        instruction = full_query.split(';')[-1]
        if 'drawer0' in instruction:
            mask[i] = 0
            
    return mask 


def filter_tasks(processed_data, raw_data):
    """
    Filter the tasks that are not suitable for the environment based on hand-crafted rules.
    """
    filter_funcs = [filter_drawer0]
    mask = np.ones((len(processed_data)))
    for func in filter_funcs:
        mask = func(processed_data, raw_data, mask)
        
    filtered_processed_data = [processed_data[i] for i in range(len(processed_data)) if mask[i] == 1]
    filtered_raw_data = [raw_data[i] for i in range(len(raw_data)) if mask[i] == 1]
    return filtered_processed_data, filtered_raw_data

processed_file_path = os.path.join(pkg_root, processed_file)
raw_file_path = os.path.join(pkg_root, raw_file)

with open(processed_file_path, 'r') as f:
    processed_data = json.load(f)
with open(raw_file_path, 'r') as f:
    raw_data = json.load(f)

# filter tasks that are not suitable for the environment
filtered_processed_data, filtered_raw_data = filter_tasks(processed_data, raw_data)

## Pick and Place

In [ ]:
env.reset()

In [ ]:
pose_msg = env.parse_grasp_pose("apple")

In [ ]:
rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)

In [ ]:
env.open_gripper()

In [ ]:
env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)

In [ ]:
env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)

In [ ]:
env.close_gripper(width=0.05, force=30)

In [ ]:
env.attach_object("apple")

In [ ]:
place_pose = env.parse_place_pose(object_name="apple", receptacle_name="white_ceramic_plate")

In [ ]:
env.move_to_pose(place_pose)
env.open_gripper()

In [ ]:
env.detach_object("apple")

## Open Drawer

In [ ]:
jr.publish('/rviz/moveit/move_marker/goal_panda_hand_tcp', PoseStamped)

In [ ]:
env.reset()
# env.open_gripper()

In [ ]:
# env.get_obj_name_list()
# [bbox.object_id for bbox in env.gazebo_gt_bboxes]
env.get_3d_bbox("cabinet.handle_0")

In [ ]:
grasp_pose = env.parse_grasp_pose(object_name="cabinet.handle_0")
print(grasp_pose)

In [ ]:
env.publish_goal_to_marker(grasp_pose)

In [ ]:
env.grasp(grasp_pose)
# env.move_to_pose(grasp_pose)

In [ ]:
env.close_gripper()

In [ ]:
# generate a horizontal trajectory to open the drawer
grasp_position = np.array([grasp_pose.position.x, grasp_pose.position.y, grasp_pose.position.z])
pull_position = grasp_position + np.array([0.2, 0, 0]).astype(float)
pull_pose = Pose(position=Point(*pull_position), orientation=grasp_pose.orientation)

In [ ]:
env.move_to_pose(pull_pose)

In [ ]:
env.open_gripper()

In [ ]:
pose_msg = env.parse_grasp_pose(object_name="apple")

rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)
 
env.open_gripper()
 
# env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)
# env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)
env.close_gripper(width=0.05, force=30)
env.attach_object("apple")


In [ ]:
env.reset()